Наступний блокнот був автоматично згенерований GitHub Copilot Chat і призначений лише для початкового налаштування


# Вступ до інженерії підказок
Інженерія підказок — це процес створення та оптимізації підказок для завдань обробки природної мови. Вона включає вибір відповідних підказок, налаштування їх параметрів і оцінку результатів. Інженерія підказок має велике значення для досягнення високої точності та ефективності моделей NLP. У цьому розділі ми розглянемо основи інженерії підказок на прикладі моделей OpenAI.


### Вправа 1: Токенізація
Ознайомтеся з токенізацією за допомогою tiktoken — швидкого токенізатора з відкритим кодом від OpenAI.
Більше прикладів можна знайти у [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst).


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Вправа 2: Перевірте налаштування ключа OpenAI API

Запустіть наведений нижче код, щоб переконатися, що ваш OpenAI endpoint налаштований правильно. Код просто виконує простий базовий запит і перевіряє відповідь. Введення `oh say can you see` має завершитися приблизно як `by the dawn's early light..`


In [ ]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

## Updated
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt)
print(response)

### Вправа 3: Вигадки
Дослідіть, що відбувається, коли ви просите LLM надати відповіді на запит про тему, яка може не існувати, або про теми, про які вона може не знати, оскільки вони були поза межами її попередньо навченого набору даних (більш сучасні). Подивіться, як змінюється відповідь, якщо ви спробуєте інший запит або іншу модель.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt)
print(response)

### Вправа 4: На основі інструкції
Використовуйте змінну "text" для основного змісту,
а змінну "prompt" — щоб дати інструкцію, пов’язану з цим змістом.

Тут ми просимо модель підсумувати текст для учня другого класу


In [ ]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt)
print(response)

### Вправа 5: Складний запит
Спробуйте запит, який містить повідомлення системи, користувача та асистента
Система задає контекст для асистента
Повідомлення користувача та асистента створюють багатокроковий контекст розмови

Зверніть увагу, що особистість асистента у системному контексті встановлена як "саркастична".
Спробуйте використати інший контекст особистості. Або спробуйте іншу серію вхідних/вихідних повідомлень


In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

### Вправа: Досліджуйте свою інтуїцію
Наведені вище приклади дають вам шаблони, які можна використовувати для створення нових підказок (простих, складних, інструкцій тощо) — спробуйте придумати інші вправи, щоб дослідити деякі з ідей, про які ми говорили, наприклад, приклади, підказки та інше.



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, звертаємо вашу увагу, що автоматичний переклад може містити помилки або неточності. Оригінальний документ рідною мовою слід вважати авторитетним джерелом. Для отримання критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильне тлумачення, що виникли внаслідок використання цього перекладу.
